## **MACHINE LEARNING PRODUCTS (MLOps)**


***En este cuaderno jupyter se encuentra la aplicación de ML dentro del producto minimo viable de nuestro proyecto - "Sistema de Recomendacion del Mundial del 2026" en donde utilizaremos la data de Google Maps***

In [1]:
# Librerias
import mysql.connector
from mysql.connector import Error
from sqlalchemy import create_engine
from dotenv import load_dotenv
import pandas as pd
import os

In [2]:
# La data se encuentra en AWS vamos a conectar
# Cargamos el archivo .env
load_dotenv()

# Acceder a las variables entorno
password_bd = os.getenv("PASS_BD")
user_bd = os.getenv("USER_BD")
host_name = os.getenv("HOST_NAME")
name_bd = os.getenv("NAME_BD")

# Conectamos la base de datos


def BD_connection(host_name, user_bd, user_pass, name_bd):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_bd,
            passwd=user_pass,
            database=name_bd,
            port=3306,
            connection_timeout=300
        )
        print("MYSQL DATABASE connection succesful")
    except Error as err:
        print(f"Error: '{err}'")
    return connection

In [3]:
connection = BD_connection(host_name, user_bd, password_bd, name_bd)

MYSQL DATABASE connection succesful


In [4]:
# verifiquemos la conexion realizando una consulta simple
if connection:
    cursor = connection.cursor()
    cursor.execute("SELECT DATABASE()")
    db = cursor.fetchone()
    print(f"Conectado a la base de datos: {db[0]}")

Conectado a la base de datos: Google


In [5]:
# Funcion para obtener las tablas disponibles
def show_tables(connection):
    cursor = connection.cursor()
    cursor.execute("SHOW TABLES")
    tables = cursor.fetchall()
    for table in tables:
        print(table)

In [6]:
# Corremos la query
connection = BD_connection(host_name, user_bd, password_bd, name_bd)
show_tables(connection)

MYSQL DATABASE connection succesful
('sites',)
('users_review',)
('users_review2',)


In [7]:
# Funcion para describir la estructura de una tabla
def describe_table(connection, table_name):
    cursor = connection.cursor()
    query = f"DESCRIBE {table_name}"
    cursor.execute(query)
    columns = cursor.fetchall()
    for column in columns:
        print(column)

In [8]:
describe_table(connection, 'users_review')

('id_user', b'text', 'YES', '', None, '')
('name', b'text', 'YES', '', None, '')
('time_comment', b'datetime', 'YES', '', None, '')
('text_comment', b'text', 'YES', '', None, '')
('rating', b'double', 'YES', '', None, '')
('time_resp', b'datetime', 'YES', '', None, '')
('text_resp', b'text', 'YES', '', None, '')
('gmap_id', b'text', 'YES', '', None, '')
('state', b'text', 'YES', '', None, '')


In [9]:
# Vamos a utilizar esta funcion para correr queries
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query hecha con éxito")
    except Error as err:
        print("Error", err)

In [10]:
def execute_query(connection, query, fetch_data=False):
    """
    Ejecuta una consulta SQL en la base de datos conectada.

    Args:
    - connection: Conexión a la base de datos.
    - query: Consulta SQL a ejecutar.
    - fetch_data: Booleano para indicar si se deben recuperar los datos de la consulta.

    Returns:
    - Resultados de la consulta si fetch_data es True. None en caso contrario.
    """
    cursor = connection.cursor(buffered=True)  # Crear un cursor con buffering
    try:
        cursor.execute(query)  # Ejecutar la consulta

        if fetch_data:
            resultados = cursor.fetchall()  # Recuperar todos los resultados
            return resultados
        else:
            connection.commit()  # Confirmar la transacción
            print("Query hecha con éxito")

    except Error as err:
        print("Error:", err)  # Imprimir cualquier error ocurrido
    finally:
        cursor.close()  # Cerrar el cursor
        if connection.unread_result:
            connection.next_result()  # Limpiar cualquier resultado no leído
        connection.commit()  # Asegurar que cualquier resultado pendiente sea limpiado

In [11]:
# Verificar la conexión
if connection:
    print("Conexión exitosa a la base de datos")

    # Contar registros en `users_review`
    query = "SELECT COUNT(*) FROM users_review"
    resultado = execute_query(connection, query, fetch_data=True)

    if resultado:
        print(f"Número de registros en 'users_review': {resultado[0][0]}")

    # Obtener promedio de calificaciones
    query = "SELECT AVG(rating) FROM users_review"
    resultado = execute_query(connection, query, fetch_data=True)

    if resultado:
        print(f"Promedio de calificaciones: {resultado[0][0]}")

Conexión exitosa a la base de datos
Número de registros en 'users_review': 2646726
Promedio de calificaciones: 4.2493117912469645


In [12]:
describe_table(connection, 'sites')

('name', b'text', 'YES', '', None, '')
('address', b'text', 'YES', '', None, '')
('gmap_id', b'text', 'YES', '', None, '')
('description', b'text', 'YES', '', None, '')
('latitude', b'double', 'YES', '', None, '')
('longitude', b'double', 'YES', '', None, '')
('category', b'text', 'YES', '', None, '')
('avg_rating', b'double', 'YES', '', None, '')
('num_of_reviews', b'bigint', 'YES', '', None, '')
('price', b'text', 'YES', '', None, '')
('url', b'text', 'YES', '', None, '')
('stadium_cercano', b'text', 'YES', '', None, '')
('distancia_millas', b'double', 'YES', '', None, '')
('CleanedCategory', b'varchar(255)', 'YES', '', None, '')
('MainCategory', b'varchar(255)', 'YES', '', None, '')


In [13]:
# Verificar la conexión
if connection:
    print("Conexión exitosa a la base de datos")

    # Contar registros en `sites`
    query = "SELECT COUNT(*) FROM sites"
    resultado = execute_query(connection, query, fetch_data=True)

    if resultado:
        print(f"Número de registros en 'sites': {resultado[0][0]}")

    # Obtener promedio de calificaciones
    query = "SELECT AVG(avg_rating) FROM sites"
    resultado = execute_query(connection, query, fetch_data=True)

    if resultado:
        print(f"Promedio de calificaciones: {resultado[0][0]}")

Conexión exitosa a la base de datos
Número de registros en 'sites': 408988
Promedio de calificaciones: 4.238985006895051


In [14]:
# Crear el engine de SQLAlchemy usando la conexion MySQL
def create_sqlalchemy_engine(host_name, user_bd, password_bd, name_bd):
    url = f"mysql+mysqlconnector://{user_bd}:{password_bd}@{host_name}/{name_bd}"
    return create_engine(url)

In [15]:
# Crear el engine
engine = create_sqlalchemy_engine(host_name, user_bd, password_bd, name_bd)

In [16]:
# Consultar SQL para obtener la tabla 'users_review' y 'sites'
query_users_review = "SELECT * FROM users_review"
query_sites = "SELECT * FROM sites"

In [17]:
# Cargamos los datos en un Dataframe
users_review = pd.read_sql(query_users_review, engine)
sites = pd.read_sql(query_sites, engine)

In [18]:
# Punto de reinicio
df_review = pd.DataFrame(users_review)
df_sites = pd.DataFrame(sites)

In [19]:
df_review.head()

,id_user,name,time_comment,text_comment,rating,time_resp,text_resp,gmap_id,state
0,1.0852584826078757e20,Rebecca Mireles,2024-05-20 02:43:00,Great Place,4.1,1900-01-01,No response from the establishment,0x80c2c7b4a08146a5,California
1,1.0852584826078757e20,Rebecca Mireles,2024-05-20 02:43:00,Great Place,4.4,1900-01-01,No response from the establishment,0x80c2c7ca4b56f82f,California
2,1.0852584826078757e20,Rebecca Mireles,2024-05-20 02:43:00,Great Place,4.2,1900-01-01,No response from the establishment,0x80c2b6ad53a92cf7,California
3,1.0852584826078757e20,Rebecca Mireles,2024-05-20 02:43:00,Great Place,4.3,1900-01-01,No response from the establishment,0x80c2b37ffd8b704b,California
4,1.0852584826078757e20,Rebecca Mireles,2024-05-20 02:43:00,Great Place,4.3,1900-01-01,No response from the establishment,0x80c2c7d67ce0e44b,California


In [20]:
df_sites.head()

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,url,stadium_cercano,distancia_millas,CleanedCategory,MainCategory
0,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,34.018891,-118.215290,['Textile exporter'],4.5,6,None,https://www.google.com/maps/place//data=!4m2!3...,SoFi Stadium,8.407156,None,None
1,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,['Korean restaurant'],4.4,18,None,https://www.google.com/maps/place//data=!4m2!3...,SoFi Stadium,7.709565,None,None
2,Nova Fabrics,"Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...",0x80c2c89923b27a41:0x32041559418d447,None,34.023669,-118.232930,['Fabric store'],3.3,6,None,https://www.google.com/maps/place//data=!4m2!3...,SoFi Stadium,7.775596,None,None
3,Nobel Textile Co,"Nobel Textile Co, 719 E 9th St, Los Angeles, C...",0x80c2c632f933b073:0xc31785961fe826a6,None,34.036694,-118.249421,['Fabric store'],4.3,7,None,https://www.google.com/maps/place//data=!4m2!3...,SoFi Stadium,7.706895,None,None
4,Matrix International Textiles,"Matrix International Textiles, 1363 S Bonnie B...",0x80c2cf163db6bc89:0x219484e2edbcfa41,None,34.015505,-118.181839,['Fabric store'],3.5,6,None,https://www.google.com/maps/place//data=!4m2!3...,SoFi Stadium,9.973596,None,None


### Aplicar ML para calcular una nueva columna llamada 'calificacion_ajustada' que se base en el promedio del rating ('avg_rating') y la cantidad de reviews ('num_of_reviews')

#### Consideraciones para elegir *k*: 

Para determinar el valor de k en el contexto de ajustar la calificación según el número de reseñas (num_of_reviews), es útil considerar la distribución de este dato en el conjunto de datos. Dado que el número de reseñas tiene un rango amplio y presenta un sesgo positivo (con una alta kurtosis y sesgo), esto sugiere que hay muchos negocios con pocas reseñas, mientras que unos pocos tienen un número muy alto de reseñas.

In [21]:
from sklearn.metrics import mean_squared_error

# Función para calcular el MSE para un valor de k


def calcular_mse_k(k):
    df_sites['calificacion_ajustada'] = (
        df_sites['avg_rating'] * df_sites['num_of_reviews'] /
        (df_sites['num_of_reviews'] + k)
    )
    df_sites['calificacion_ajustada'] = df_sites['calificacion_ajustada'].clip(
        lower=1.0, upper=5.0)
    return mean_squared_error(df_sites['calificacion_ajustada'], df_sites['avg_rating'])


# Valores de k a probar
valores_k = [5, 10, 20, 30, 50, 100]
resultados = {}

# Evaluar cada valor de k
for k in valores_k:
    mse = calcular_mse_k(k)
    resultados[k] = mse

# Mostrar resultados
print(resultados)

{5: 2.3166022361495875, 10: 3.7628772344043644, 20: 5.5366346558131045, 30: 6.590332041132454, 50: 7.8231021030353425, 100: 9.276346697289881}


Recomendación:

Utilizar k=5, ya que tiene el menor MSE, lo que indica que la calificación ajustada está más alineada con la calificación promedio (avg_rating). Este valor de k proporciona un buen balance entre el promedio de calificaciones y la cantidad de reseñas, permitiendo que los negocios con menos reseñas no sean penalizados excesivamente.

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Definir el valor de k para el cálculo de calificacion_ajustada
k = 5

# Calcular la calificacion_ajustada
df_sites['calificacion_ajustada'] = (
    df_sites['avg_rating'] * df_sites['num_of_reviews'] /
    (df_sites['num_of_reviews'] + k)
)

# Normalizar la calificacion_ajustada para que esté en el rango de 1.0 a 5.0
df_sites['calificacion_ajustada'] = df_sites['calificacion_ajustada'].clip(
    lower=1.0, upper=5.0)

# Seleccionar características (features) y la variable objetivo (target)
X = df_sites[['avg_rating', 'num_of_reviews']]
y = df_sites['calificacion_ajustada']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Entrenar el modelo de regresión lineal
model = LinearRegression()
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Predecir la calificacion ajustada para todo el dataframe
df_sites['calificacion_ajustada_predicha'] = model.predict(X)

# Normalizar las predicciones para que estén en el rango de 1.0 a 5.0
df_sites['calificacion_ajustada_predicha'] = df_sites['calificacion_ajustada_predicha'].clip(
    lower=1.0, upper=5.0)

# Redondear la calificacion_ajustada_predicha a 4 dígitos decimales
df_sites['calificacion_ajustada_predicha'] = df_sites['calificacion_ajustada_predicha'].round(
    4)

# Mostrar el dataframe con las nuevas columnas
df_sites[['name', 'avg_rating', 'num_of_reviews',
          'calificacion_ajustada', 'calificacion_ajustada_predicha']]

Mean Squared Error: 0.8359213972976306


,name,avg_rating,num_of_reviews,calificacion_ajustada,calificacion_ajustada_predicha
0,City Textile,4.5,6,2.454545,3.1868
1,San Soo Dang,4.4,18,3.443478,3.1410
2,Nova Fabrics,3.3,6,1.800000,2.4479
3,Nobel Textile Co,4.3,7,2.508333,3.0649
4,Matrix International Textiles,3.5,6,1.909091,2.5710
...,...,...,...,...,...
408983,Hotel Ändra Seattle MGallery Collection,4.5,1,1.000000,3.1802
408984,Fairmont Olympic Hotel - Seattle,4.6,3,1.725000,3.2444
408985,Seatac Inn and Airport Parking,3.2,613,3.174110,3.1866
408986,"Reside Seattle Downtown, a Wyndham Residence",4.0,36,3.512195,2.9184


El código implementa un modelo de regresión lineal utilizando scikit-learn para predecir la calificación ajustada de negocios basada en su promedio de calificaciones (`avg_rating`) y el número de reseñas (`num_of_reviews`). Primero, se calcula una calificación ajustada utilizando una fórmula que incluye un parámetro \( k \) para suavizar el impacto de reseñas escasas. Luego, se normaliza este valor para que esté en el rango de 1.0 a 5.0. Los datos se dividen en conjuntos de entrenamiento y prueba, y el modelo se entrena sobre el conjunto de entrenamiento. Posteriormente, se evalúa su rendimiento mediante el cálculo del error cuadrático medio (MSE) en el conjunto de prueba. Finalmente, se utilizan las predicciones del modelo para agregar una columna de calificación ajustada predicha al DataFrame, asegurando que estas también se mantengan dentro del rango deseado.

El Mean Squared Error (MSE) es una medida que evalúa la calidad de un modelo de predicción. Representa la media de los cuadrados de las diferencias entre los valores reales y los valores predichos. En este caso, un MSE de 0.8359 significa que, en promedio, las predicciones del modelo se desvían de los valores reales en aproximadamente 0.84 (al elevar al cuadrado las diferencias, se penalizan más los errores grandes).

In [23]:
df_sites

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,url,stadium_cercano,distancia_millas,CleanedCategory,MainCategory,calificacion_ajustada,calificacion_ajustada_predicha
0,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,34.018891,-118.215290,['Textile exporter'],4.5,6,None,https://www.google.com/maps/place//data=!4m2!3...,SoFi Stadium,8.407156,None,None,2.454545,3.1868
1,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,['Korean restaurant'],4.4,18,None,https://www.google.com/maps/place//data=!4m2!3...,SoFi Stadium,7.709565,None,None,3.443478,3.1410
2,Nova Fabrics,"Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...",0x80c2c89923b27a41:0x32041559418d447,None,34.023669,-118.232930,['Fabric store'],3.3,6,None,https://www.google.com/maps/place//data=!4m2!3...,SoFi Stadium,7.775596,None,None,1.800000,2.4479
3,Nobel Textile Co,"Nobel Textile Co, 719 E 9th St, Los Angeles, C...",0x80c2c632f933b073:0xc31785961fe826a6,None,34.036694,-118.249421,['Fabric store'],4.3,7,None,https://www.google.com/maps/place//data=!4m2!3...,SoFi Stadium,7.706895,None,None,2.508333,3.0649
4,Matrix International Textiles,"Matrix International Textiles, 1363 S Bonnie B...",0x80c2cf163db6bc89:0x219484e2edbcfa41,None,34.015505,-118.181839,['Fabric store'],3.5,6,None,https://www.google.com/maps/place//data=!4m2!3...,SoFi Stadium,9.973596,None,None,1.909091,2.5710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408983,Hotel Ändra Seattle MGallery Collection,None,0x5490154c9e0ff655,No description,47.613396,-122.340117,Hotel,4.5,1,None,https://www.google.com/maps/place/Hotel+%C3%84...,Lumen Field,1.318614,None,None,1.000000,3.1802
408984,Fairmont Olympic Hotel - Seattle,None,0x54906ab3fa957b7f,No description,47.608013,-122.333870,Hotel,4.6,3,None,https://www.google.com/maps/place/Fairmont+Oly...,Lumen Field,0.892180,None,None,1.725000,3.2444
408985,Seatac Inn and Airport Parking,None,0x54905cac5bec0d8f,No description,47.448993,-122.296090,Hotel,3.2,613,None,https://www.google.com/maps/place/Seatac+Inn+a...,Lumen Field,10.236473,None,None,3.174110,3.1866
408986,"Reside Seattle Downtown, a Wyndham Residence",None,0x54906b469991ae5f,No description,47.610114,-122.340744,Hotel,4.0,36,None,https://www.google.com/maps/place/Reside+Seatt...,Lumen Field,1.115113,None,None,3.512195,2.9184


### Añadir las nuevas columnas a la base de datos 'Google' en la tabla 'sites' ('calificacion_ajustada' y 'calificacion_ajustada_predicha')